In [2]:
import yfinance as yf
# Remote data access for pandas
import pandas_datareader as webreader
# Mathematical functions 
import math 
# Fundamental package for scientific computing with Python
import numpy as np 
# Additional functions for analysing and manipulating data
import pandas as pd 
# Date Functions
from datetime import date, timedelta
# This function adds plotting functions for calender dates
from pandas.plotting import register_matplotlib_converters
# Important package for visualization - we use this to plot the market data
import matplotlib.pyplot as plt 
# Formatting dates
import matplotlib.dates as mdates
# Packages for measuring model performance / errors
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Tools for predictive data analysis. We will use the MinMaxScaler to normalize the price data 
from sklearn.preprocessing import MinMaxScaler 
# Deep learning library, used for neural networks
from keras.models import Sequential 
# Deep learning classes for recurrent and regular densely-connected layers
from keras.layers import LSTM, Dense
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.express as px
from IPython.display import display
import plotly.graph_objects as go
from statistics import mean
# Setting the timeframe for the data extraction

today = date.today()
date_today = today.strftime("%Y-%m-%d")
date_start = '2015-01-01'

# Getting Stock Details
stockname = input('Enter Stock name : ')
symbol = input('Enter Stock symbol : ')
days = int(input('Enter the number of days value you wish to predict [10  20  30] :'))
#look_back = 90
df = yf.Ticker(symbol)
df = df.history(start=date_start,  end=today)
df = pd.DataFrame(df)
df = df.drop(['Dividends', 'Stock Splits'], axis = 1)
# Taking a look at the shape of the dataset
#print(df.shape)
print('Latest 5 days values of',stockname)
display(df.tail(5))

def OCHL(df,days,z):
#Plotting close price to dates
    x = df.index
    y = df[z]
    #print(z,'Price :')
    fig = px.line(df, x=x, y=y, title= z+' price of '+stockname)
    fig.show()

    # Splitting the Data
    # Create a new dataframe with only the Close column and convert to numpy array
    data = df.filter([z])
    npdataset = data.values

    # Get the number of rows to train the model on 75% of the data 
    training_data_length = math.ceil(len(npdataset) * 0.75)

    # Transform features by scaling each feature to a range between 0 and 1
    mmscaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = mmscaler.fit_transform(npdataset)
    #print(scaled_data)
    train_data = scaled_data[0:training_data_length, :]
    #print(len(train_data))

    # Creating the input shape.
    # Create a scaled training data set
    train_data = scaled_data[0:training_data_length, :]

    # Split the data into x_train and y_train data sets
    x_train = []
    y_train = []
    trainingdatasize = len(train_data) 
    for i in range(100, trainingdatasize):
        x_train.append(train_data[i-100: i, 0]) #contains 100 values 0-100
        y_train.append(train_data[i, 0]) #contains all other values

    # Convert the x_train and y_train to numpy arrays
    a = x_train
    b = y_train
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    #print(x_train.shape)
    #print(y_train.shape)

    # Configure the neural network model
    model = Sequential()

    # Model with 100 Neurons - inputshape = 100 Timestamps
    model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1))) 
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Training the model
    #model.fit(x_train, y_train, epochs=60)
    lst=[]
    #model.fit(x_train, y_train, epochs=10)
    history = model.fit(x_train, y_train, epochs=100)
    lst=list(history.history['loss'])

    all_acc_of_epochs = []
    for i in range(len(lst)):
        acc = 100-lst[i]
        #print("Total accuracy for %s epoch is %s" % (i+1, acc))
        all_acc_of_epochs.append(acc)
    #print(all_acc_of_epochs)
    avg_of_all_epochs = mean(all_acc_of_epochs)
    print('Total average accuracy of all epochs :',avg_of_all_epochs)

    # Create a new array containing scaled test values
    test_data = scaled_data[training_data_length - 100:, :]

    # Create the data sets x_test and y_test
    x_test = []
    y_test = npdataset[training_data_length:, :]
    for i in range(100, len(test_data)):
        x_test.append(test_data[i-100:i, 0])

    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data, so that we get an array with multiple test datasets
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    #print(len(x_test))

    #Get the predicted values and inverse the scaling
    predictions = model.predict(x_test)
    predictions = mmscaler.inverse_transform(predictions)
    #print(predictions)
    #print(len(predictions))

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(predictions, y_test)
    print('MAE: ' + str(round(mae, 1)))
    
    # Calculate the root mean squarred error (RMSE)
    mse = mean_squared_error(predictions,y_test)
    rmse = np.sqrt(mse)
    print('RMSE: ' + str(round(rmse, 1)))


    # The date from which on the date is displayed
    display_start_date = "2018-01-01" 

    # Add the difference between the valid and predicted prices
    train = data[:training_data_length + 1]
    valid = data[training_data_length:]
    valid.insert(1, "Predictions", predictions, True)
    valid.insert(1, "Difference", valid["Predictions"] - valid[z], True)

    # Zoom in to a closer timeframe
    valid = valid[valid.index > display_start_date]
    train = train[train.index > display_start_date]

    xt = train.index
    yt = train[[z]]
    xv = valid.index
    yv = valid[[z, "Predictions"]]


    fig = px.line(df,x=x, y=y, title = 'Predictied value v/s actual value of '+z+' Price '+stockname)
    fig.add_scatter(x=xv, y=yv['Predictions'],name = 'Predicted Values')
    fig.show()

    fut_pred = df
    open_data = fut_pred.tail(100)
    open_data = open_data[z].values
    open_data = open_data.reshape((-1))
    #print(open_data)
    if(days == 10):
        look_back = 50
    elif(days == 20):
        look_back = 70
    elif(days == 30):
        look_back = 90
    num_prediction = days
    forecast = predict(num_prediction, model,open_data,look_back)
    forecast_dates = predict_dates(num_prediction,fut_pred)
    f = pd.DataFrame(forecast)
    f_d = pd.DataFrame(forecast_dates)
    if(days == 10):
        f = f.head(11)
        f_d = f_d.head(11)
    elif(days == 20):
        f = f.head(21)
        f_d = f_d.head(21)
    elif(days == 30):
        f = f.head(31)
        f_d = f_d.head(31)


    df_1 = df.index

    # Printing the predicted values of next 10 days
    f.columns = [z]
    f_d.columns = ['Date']

    fp = pd.concat([f_d, f], axis=1)
    fp = pd.DataFrame(fp)

    fp = fp.set_index('Date')

    a = pd.DataFrame()
    b = pd.DataFrame()

    a = data.tail(10)
    b = fp

    finals = a.append(b)
    
    df_1 = df.index

    
    final = finals.values.tolist()

    #Analysis for future predicttions
    increase = 0
    decrease = 0
    neutral = 0
    for i, j in enumerate(final[:-1]):
        if j < final[i+1]:
            increase = increase + 1
        if j > final[i+1]:
            decrease = decrease + 1
        if j == final[i+1]:
            neutral = neutral + 1
    #print(increase)

    Faith = -1
    if(increase>decrease and increase>neutral):
        print('Stock Faith is Positive on',z,'Price.')
        Faith = 1
    elif(decrease>increase and decrease>neutral):
        print('Stock Faith is Negative on',z,'Price.')
        Faith = -1
    else:
        print('Stock Faith is Neutral on',z,'Price.')
        #Faith_of_Open = 0
    return finals,Faith
        

def predict(num_prediction,model,data,look_back):
    prediction_list = data[-look_back:]
    #print(prediction_list)
    #print(len(prediction_list))
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        #print(x)
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        #print(out)
        prediction_list = np.append(prediction_list, out)
        #print(prediction_list)
    prediction_list = prediction_list[-look_back:]
    #pediction_list = prediction_list.reshape((-1))    
    return prediction_list

def predict_dates(num_prediction,fut_pred):
    last_date = fut_pred.index
    last_date = last_date.values[-1]
    prediction_dates = pd.date_range(date_today, periods=num_prediction+1).tolist()
    return prediction_dates

    
def senti(symbol):
    web_url = 'https://finviz.com/quote.ashx?t='

    news_tables = {}
    tickers = symbol
    url = web_url + tickers
    req = Request(url=url,headers={"User-Agent":"Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(id='news-table')
    news_tables[tickers] = news_table

    #print(url)

    #print(news_tables)

    data = news_tables[tickers]
    data_tr = data.findAll('tr')

    for x, table_row in enumerate(data_tr):
        a_text = table_row.a.text
        td_text = table_row.td.text
        #print(a_text)
        #print(td_text)
        if x == 3:
            break

    news_list = []

    for file_name, news_table in news_tables.items():
        for i in news_table.findAll('tr'):

            text = i.a.get_text() 

            date_scrape = i.td.text.split()

            if len(date_scrape) == 1:
                time = date_scrape[0]

            else:
                date = date_scrape[0]
                time = date_scrape[1]

            tick = file_name.split('_')[0]

            news_list.append([tick, date, time, text])


    #news_list

    vader = SentimentIntensityAnalyzer()

    columns = ['ticker', 'date', 'time', 'headline']

    news_df = pd.DataFrame(news_list, columns=columns)

    scores = news_df['headline'].apply(vader.polarity_scores).tolist()

    scores_df = pd.DataFrame(scores)

    news_df = news_df.join(scores_df, rsuffix='_right')

    news_df['date'] = pd.to_datetime(news_df.date).dt.date

    news_df = pd.DataFrame(news_df)

    #display(news_df)

    plt.rcParams['figure.figsize'] = [10, 6]

    mean_scores = news_df.groupby(['ticker','date']).mean()

    mean_scores = mean_scores.unstack()

    mean_scores = mean_scores.xs('compound', axis="columns").transpose()

    m = pd.DataFrame()
    m = news_df.groupby(['date']).mean()
    #m

    x = m.index
    y = m[['neg']]
    #fig = px.bar(m, x=m.index, y=["neg", "neu", "pos"], title="Sentiment Analysis on news of "+symbol)
    #fig.show()

    a = pd.DataFrame()
    a = m[['compound']]
    #print(a)

    a = a.values.tolist()

    final = a
    #print(final)
    increase = 0
    decrease = 0
    neutral = 0
    for i, j in enumerate(final[:-1]):
        if i > 0:
            increase = increase + 1
        if i < 0:
            decrease = decrease + 1
        if i == 0:
            neutral = neutral + 1
    #print(increase)

    Faith_of_Senti = -1
    if(increase>decrease and increase>neutral):
        print('\nStock Faith is Positive on analysis of news of the stock.')
        Faith_of_Senti = 1
    elif(decrease>increase and decrease>neutral):
        print('\nStock Faith is Negative on analysis of news of the stock.')
        Faith_of_Senti = -1
    else:
        print('\nStock Faith is Neutral on analysis of news of the stock.')
        #Faith_of_Senti = 0
    return m,news_df,Faith_of_Senti

Open_Val,O = OCHL(df,days,'Open')
Close_Val,C = OCHL(df,days,'Close')
High_Val,H = OCHL(df,days,'High')
Low_Val,L = OCHL(df,days,'Low')
m,news,S = senti(symbol)


print('\n',stockname, 'predicted values with previous 10 days actual values :')

x = pd.concat([Open_Val, Close_Val, High_Val, Low_Val], axis=1, ignore_index=True)
x.columns = ['Open', 'Close', 'High','Low']
display(x)

trace1 = go.Scatter(
    x = x.index,
    y = x['Open'],
    mode = 'lines',
    name = 'Open'
)
trace2 = go.Scatter(
    x = x.index,
    y = x['Close'],
    mode = 'lines',
    name = 'Close'
)
trace3 = go.Scatter(
    x = x.index,
    y = x['High'],
    mode = 'lines',
    name = 'High'
)
trace4 = go.Scatter(
    x = x.index,
    y = x['Low'],
    mode = 'lines',
    name = 'Low'
)
layout = go.Layout(
    title = stockname+' Predicted values with previous 10 days actual values.',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Prices"})

fig = go.Figure(data=[trace1,trace2,trace3,trace4], layout=layout)
fig.show()

print('\nNews Analysis on ',stockname)
display(news.tail())

fig = px.bar(m, x=m.index, y=["neg", "neu", "pos"], title="Sentiment Analysis on news of "+symbol)
fig.show()


print('\nConsidering the best possible attributes of the stock : \n\t')
print('Attributes : Opening Price, Closing Price, Everyday High, Everyday Low and News on Stock')
if(O == -1 and C == -1 and H == -1 and L == -1 and S == -1):
    print('Stock Faith is Negative, may not be a good option.')
elif(O == -1 and C == -1 and H == -1 and L == -1 and S == 1):
    print('Stock Faith is Low, news of the stock is positive.')
elif(O == -1 and C == -1 and H == -1 and L == 1 and S == -1):
    print('Stock Faith is Low, everyday Low price is good.')
elif(O == -1 and C == -1 and H == -1 and L == 1 and S == 1):   
    print('Stock Faith is Low, everyday Low price and news is good.')
elif(O == -1 and C == -1 and H == 1 and L == -1 and S == -1):
    print('Stock Faith is Low, everyday High price is good.')
elif(O == -1 and C == -1 and H == 1 and L == -1 and S == 1):
    print('Stock Faith is low, everyday High price and news is good.')
elif(O == -1 and C == -1 and H == 1 and L == 1 and S == -1):
    print('Stock Faith is low, everyday High and Low price is good.')
elif(O == -1 and C == -1 and H == 1 and L == 1 and S == 1):
    print('Stock Faith is good, Opening and Closing price is low.')
elif(O == -1 and C == 1 and H == -1 and L == -1 and S == -1):
    print('Stock Faith is Low, only Closing price is good.')
elif(O == -1 and C == 1 and H == -1 and L == -1 and S == 1):
    print('Stock Faith is low, Closing price and news is good.')
elif(O == -1 and C == 1 and H == -1 and L == 1 and S == -1):
    print('Stock Faith is low, closing price and and everyday Low is good.')
elif(O == -1 and C == 1 and H == -1 and L == 1 and S == 1):
    print('Stock Faith is good, Opening price and everyday High is low.')
elif(O == -1 and C == 1 and H == 1 and L == -1 and S == -1):
    print('Stock Faith is low, Closing price and everyday High is good.')
elif(O == -1 and C == 1 and H == 1 and L == -1 and S == 1):
    print('Stock Faith is good, Opening price and everyday Low is low.')
elif(O == -1 and C == 1 and H == 1 and L == 1 and S == -1):
    print('Stock Faith is good, Opening price and news is low.')
elif(O == -1 and C == 1 and H == 1 and L == 1 and S == 1):
    print('Stock Faith is good, only Opening price is Low.')
elif(O == 1 and C == -1 and H == -1 and L == -1 and S == -1):
    print('Stock Faith is low, only Opening price is good.')
elif(O == 1 and C == -1 and H == -1 and L == -1 and S == 1):
    print('Stock Faith is low, Opening price and news is good.')
elif(O == 1 and C == -1 and H == -1 and L == 1 and S == -1):
    print('Stock Faith is low, Opening price and everyday Low is good.')
elif(O == 1 and C == -1 and H == -1 and L == 1 and S == 1):
    print('Stock Faith is good, Closing price and everyday High is low.')
elif(O == 1 and C == -1 and H == 1 and L == -1 and S == -1):
    print('Stock Faith is low, Opening price and everyday High is good.')
elif(O == 1 and C == -1 and H == 1 and L == -1 and S == 1):
    print('Stock Faith is good, Closing price and everyday Low is low.')
elif(O == 1 and C == -1 and H == 1 and L == 1 and S == -1):
    print('Stock Faith is good, Closing price and news is low.')
elif(O == 1 and C == -1 and H == 1 and L == 1 and S == 1):
    print('Stock Faith is good, only Closing price is low.')
elif(O == 1 and C == 1 and H == -1 and L == -1 and S == -1):
    print('Stock Faith is low, Opening Price and Closing price is good')
elif(O == 1 and C == 1 and H == -1 and L == -1 and S == 1):
    print('Stock Faith is good, everyday High and Low is low')
elif(O == 1 and C == 1 and H == -1 and L == 1 and S == -1):
    print('Stock Faith is good, everyday High and news is low.')
elif(O == 1 and C == 1 and H == -1 and L == 1 and S == -1):
    print('Stock Faith is good, only everyday High is low.')
elif(O == 1 and C == 1 and H == 1 and L == -1 and S == -1):
    print('Stock Faith is good, everyday Low and news is low.')
elif(O == 1 and C == 1 and H == 1 and L == -1 and S == 1):
    print('Stock Faith is good, only everyday Low is low.')
elif(O == 1 and C == 1 and H == 1 and L == 1 and S == -1):#
    print('Stock Faith is good, only news is low.')
elif(O == 1 and C == 1 and H == 1 and L == 1 and S == 1):
    print('Stock Faith is Positive, a good option.')
    

Enter Stock name : Microsoft
Enter Stock symbol : MSFT
Enter the number of days value you wish to predict [10  20  30] :20
Latest 5 days values of Microsoft


,Open,High,Low,Close,Volume
Date,,,,,
2022-09-06,256.200012,257.829987,251.940002,253.250000,21328200
2022-09-07,254.699997,258.829987,253.220001,258.089996,24126700
2022-09-08,257.510010,260.429993,254.789993,258.519989,20319900
2022-09-09,260.500000,265.230011,260.290009,264.459991,22084700
2022-09-12,265.779999,267.450012,265.160004,266.649994,18731500


Epoch 1/100
43/43 [==============================] - 8s 123ms/step - loss: 0.0037
Epoch 2/100
43/43 [==============================] - 5s 115ms/step - loss: 2.1431e-04
Epoch 3/100
43/43 [==============================] - 5s 124ms/step - loss: 2.3457e-04
Epoch 4/100
43/43 [==============================] - 5s 113ms/step - loss: 1.7217e-04
Epoch 5/100
43/43 [==============================] - 5s 111ms/step - loss: 1.6083e-04
Epoch 6/100
43/43 [==============================] - 5s 109ms/step - loss: 1.6336e-04
Epoch 7/100
43/43 [==============================] - 5s 111ms/step - loss: 1.9703e-04
Epoch 8/100
43/43 [==============================] - 5s 107ms/step - loss: 2.2092e-04
Epoch 9/100
43/43 [==============================] - 5s 108ms/step - loss: 1.5824e-04
Epoch 10/100
43/43 [==============================] - 5s 107ms/step - loss: 2.2530e-04
Epoch 11/100
43/43 [==============================] - 5s 107ms/step - loss: 1.7357e-04
Epoch 12/100
43/43 [==============================] - 5s

43/43 [==============================] - 5s 110ms/step - loss: 5.8027e-05
Epoch 96/100
43/43 [==============================] - 5s 113ms/step - loss: 6.4504e-05
Epoch 97/100
43/43 [==============================] - 5s 114ms/step - loss: 5.8354e-05
Epoch 98/100
43/43 [==============================] - 5s 115ms/step - loss: 5.8594e-05
Epoch 99/100
43/43 [==============================] - 5s 110ms/step - loss: 5.7306e-05
Epoch 100/100
43/43 [==============================] - 5s 110ms/step - loss: 4.7487e-05
Total average accuracy of all epochs : 99.99986991413999
16/16 [==============================] - 1s 34ms/step
MAE: 3.6
RMSE: 4.8


1/1 [==============================] - 0s 20ms/step
Stock Faith is Positive on Open Price.


...

1/1 [==============================] - 0s 23ms/step
Stock Faith is Neutral on Low Price.

Stock Faith is Positive on analysis of news of the stock.

 Microsoft predicted values with previous 10 days actual values :


,Open,Close,High,Low
Date,,,,
2022-08-29,265.850006,265.230011,267.399994,263.850006
2022-08-30,266.670013,262.970001,267.049988,260.660004
2022-08-31,265.390015,261.470001,267.109985,261.329987
2022-09-01,258.869995,260.399994,260.890015,255.410004
2022-09-02,261.700012,256.059998,264.739990,254.470001
2022-09-06,256.200012,253.250000,257.829987,251.940002
2022-09-07,254.699997,258.089996,258.829987,253.220001
2022-09-08,257.510010,258.519989,260.429993,254.789993
2022-09-09,260.500000,264.459991,265.230011,260.290009



News Analysis on  Microsoft


,ticker,date,time,headline,neg,neu,pos,compound
95,MSFT,2022-09-01,10:33AM,10 Best Dividend Stocks to Buy According to Th...,0.00,0.724,0.276,0.6369
96,MSFT,2022-09-01,10:13AM,Microsofts $69bn Call of Duty deal faces inves...,0.00,1.000,0.000,0.0000
97,MSFT,2022-09-01,09:55AM,Leadership expert: Meta CEO Mark Zuckerberg is...,0.00,1.000,0.000,0.0000
98,MSFT,2022-09-01,09:47AM,Microsofts $75bn Activision deal threatened wi...,0.25,0.750,0.000,-0.4588
99,MSFT,2022-09-01,08:13AM,Activision CEO still sees Microsoft deal closi...,0.00,1.000,0.000,0.0000



Considering the best possible attributes of the stock : 
	
Attributes : Opening Price, Closing Price, Everyday High, Everyday Low and News on Stock
Stock Faith is good, Closing price and everyday Low is low.
